Major Challenges:

• Cleaning data in a robust, consistent way

• Handling gaps in different parts of our data

• Handling extremely large data sets

• Structuring the data in an optimal way

• Building robust data integrity checks to ensure high quality data

open-source financial datasets such as the Fama-French factors that are publicly downloadable on Ken French's website; Yahoo Finance Data - has an API in Python that we can leverage.

In [ ]:
# Reading & Writing Files
import pandas as pd
df = pd.read_csv ('AAPL.csv', index_col = 'Date')

# unformatted data: read the file in line by line - use an indefinite loop to iterate through and parse all lines
def parse_line(line):
    # Implement your line parsing logic here
    print(line.strip())

# Open the file in read mode
with open(df, 'r') as f:
    # Read each line in the file
    line = f.readline()
    while line:
        parse_line(line)
        line = f.readline()
f.close()

In [ ]:
# Parsing Data from a Website

# if working with a website that has relatively simple formatting
import pandas as pd
dfs = pd.read_html ('https://en.wikipedia.org/wiki/Minnesota')
df = dfs[0]
df.head()


,Minnesota,Minnesota.1
0,State,State
1,.mw-parser-output .ib-settlement-cols{text-ali...,.mw-parser-output .ib-settlement-cols{text-ali...
2,"Nicknames: Land of 10,000 Lakes; North Star St...","Nicknames: Land of 10,000 Lakes; North Star St..."
3,Motto: L'Étoile du Nord (French: The Star of t...,Motto: L'Étoile du Nord (French: The Star of t...
4,"Anthem: ""Hail! Minnesota""","Anthem: ""Hail! Minnesota"""


In [ ]:
# make a request to a site and download the resulting html code
import requests
url = 'https://en.wikipedia.org/wiki/Minnesota'
r = requests.get(url)
r.status_code
#r.content

200

In [ ]:
pip install pypyodbc

  Preparing metadata (setup.py) ... done
  Created wheel for pypyodbc: filename=pypyodbc-1.3.6-py3-none-any.whl size=22858 sha256=20d522effd58cb04cff62fb409bbf2cf04d4ccfd2ecd06686e1c51eacdf1e4e3
  Stored in directory: /root/.cache/pip/wheels/5e/7f/c4/1cb1f9438e53058122c259eda71b054d96c0760b480cc88ff5
Successfully built pypyodbc


In [ ]:
# Interacting with Databases in Python
import pypyodbc
import pandas as pd

db = pyodbc.connect(driver='{SQL Server Native Client 10.0}',
                    server='192.106.0.102\instance1',
                    database='master',
                    uid='sql2008',pwd='password123')
qry = 'select * from table'
df = pd.read_sql_query (qry, db)


Common Financial Datasets

-Stock Data: price/returns data; how to look for clues about what future returns for a given equity might be? 1. look to past returns, and try to build a momentum or mean-reversion signal; 2. look at fundamental data, such as earnings; 3. algorithms that track web-clicks / parse news related to different companies

When building a model related to stocks, we work with time series data & panel data (often have a cross-section of underlying assets). When cleaning data, we check for outliers and ensure that splits, mergers, and dividends are properly handled.

-Currency Data: Historical data for a set of exchange rates. Panal data because we have a potential cross-section of exchange rates. Additional information that explains or predicts currency movements, including equity data itself, macroeconomic data, potentially data related to currency options

-Futures Data: Two types - the underlying asset itself is tradable or not. If the underlying itself is tradable, it leads to a very tight relationship between the set of futures and the underlying index that is bound by arbitrage.
A second key point regarding working with futures data is that, by the nature
of their fixed expiry, futures will eventually expire and need to be rolled in order to maintain similar levels of exposure.

When we are working with futures data, validating the data should include checking for arbitrage between the spot and futures, keeping in mind any storage costs. It should also include a rolling strategy in order to patch together a single time series for a particular future from the individual fixed expiry futures.

-Options Data: the presence of data for different strikes adds an
additional dimension to the data. In the case of options we have an entire surface for each historical date that we are analyzing. The first step in analyzing the set of options data is to identify which options are liquid enough to be included in our set. We also want to make sure that the data doesn't contain arbitrage.

-Fixed Income Data: like in futures, we have to deal with data for multiple
expiries, and therefore, a curve arises for each historical point in time.The world of fixed income can be broadly broken down into interest rate markets (yield curve) and credit markets (default rate curve)

Common Financial Data Sources

• Yahoo Finance: Perhaps the most popular free source of data for aspiring
quants. Mainly consists of stock price data with some futures data as well.
• Ken French's Website: Useful historical datasets of the returns for Fama-French factors.
• FRED: Federal Reserve Bank of St. Louis website. Contains a significant
amount of historical data on economic variables, such as GDP, employment
and credit spreads.
• Treasury.gov: Historical yield curve data for the US.
• Quandl: Contains equity market data as well as data on futures in different
asset classes.
• HistData: Contains free data from FX markets, including intra-day data.
• OptionMetrics: Contains relatively clean options and futures data for equity
and other markets. A great, but costly source of options data.
• CRSP: A broad, robust, historical database that doesn't suffer from survivorship bias. Equity prices, and other datasets available via CRSP.

Both Yahoo Finance and Quandl have API's in the Python language making
interacting with these data sources completely automated and easy to setup.
Bloomberg is another data source that many financial professionals will rely on
for retrieving data for ad-hoc analysis or for building model prototypes. Python also has API for accessing the data though a Bloomberg terminal

Handling Missing Data

-Interpolation & Filling Forward: Use a constant (generally the last good value), or a simple interpolation scheme such as linear interpolation. Benefits: simplicity
Drawbacks: potential bias, in particular the dampening or heightening of volatility

-Filling via Regression: lead to a better estimate based on what happened to other, correlated assets.

The first step in doing this is identifying related assets that have a large history that includes data on the missing data points we are looking to fill.We could use a single market index, or we could rely on multiple related securities.

To apply this method we simply need to estimate the β parameter, and the success of this method is largely dependent on the fit of regression. In cases where the explanatory power is low, this approach may lead to a dampening effect in volatility. Therefore, this method is most effective when there are longer gaps in our data and we are able to find a set of highly correlated assets.

-Filling via Bootstrapping: bootstrapping is a non-parametric technique for creating sample paths from an empirical distribution. The idea behind bootstrapping is that each realization is in actuality just a draw from some unobserved density. Bootstrapping provides a method for sampling from that density with minimal assumptions.

In practice bootstrapping involves simulating returns, and paths of asset prices by scrambling historical returns data, or sampling various historical returns and creating series of them in random order. In the context of missing data, bootstrapping enables us to create a synthetic dataset that is statistically equivalent to the dataset with missing data. We can then use the synthetic dataset instead.

• Create a matrix of historical returns for the universe of assets.
• Generate a sequence of random integers between 1 and N, where N is the
number of historical days of data.
• Look up the return data for all assets that corresponds to that randomly chosen date.
• Apply these returns to the entire asset universe.
• Continue this for many time steps until the end of a path is reached.

Each time this process is followed, a new bootstrapped path is created.When filling missing data however, we simply want to create a single path that is statistically equivalent, and as a result do not need to repeat the process. Instead we can take the path that results from the process above and substitute it for the period with missing data.

The main benefit of bootstrapping is its non-parametric nature and its ability to preserve the market correlation structure and other properties.

-Filling via K-Nearest Neighbor

At a high level, the K-Nearest Neighbor works by locating the points that are the most similar to the point it is trying to estimate. It then uses an average of those closest points, or nearest neighbors, in order to make a prediction. In the case of k = 1 simply finds the closest point and predicts that the value will be equal to what it was at this point. In the case of k > 1, the closest k points will be found, and an average of their values will be used as the prediction.

For example, in the context of filling in missing data, if we are trying to estimate the most likely the return of a given commodity, say Oil, and we are trying to do so using only the broader commodity index BCOM, then the KNN algorithm would proceed by finding the k closest returns to the current BCOM index return, and averaging the return of Oil over those k periods. The average of these k returns would then be the estimate we would use to fill in our missing data for that point.

Outlier Detection

-Single vs. Multi-Variate Outlier Detection: looking for return that appeared suspicious

-Plotting

-Standard Deviation: examine the data beyond a certain standard deviation threshold. For example, we might choose to isolate any data that is beyond a three standard deviation event and confirm that these data don't appear to be errors

-Density Analysis: similar to looking at the standard deviation, however, it is reasonable to expect that their might also be clues in the higher moments, such as skewness and kurtosis as well.
1. outliers in the histogram; 2. what the distribution of our data should closely resemble; 3. the skewness and kurtosis of our dataset can help identify the presence of outliers.

-Distance from K-Nearest Neighbor: one characterization of outliers is that they are far removed from the data. We can use the K-Nearest Neighbor algorithm to find a set of one or more close data points, and measure the distance between the point and its neighbors. Points with large distances from their neighbors are then natural outlier candidates, and should be examined
more thoroughly.